# Dummy

## Lab: Survival Analysis

In this lab, we perform survival analyses on three separate data
sets. In the first section we analyze the `BrainCancer`
data .<br />
Next, we examine the `Publication` data {islr}`from Section~\\ref{sec:pub};`. Finally, we explore explores
a simulated call center data set.

We begin by importing some of our libraries at this top
level. This makes the code more readable, as scanning the first few
lines of the notebook tell us what libraries are used in this
notebook.



In [ ]:
%%R
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Brain Cancer Data

We begin with the `BrainCancer` data set, which is part of the `ISLP`.



In [ ]:
%%R
from ISLP import load_data
BrainCancer = load_data('BrainCancer')
BrainCancer.columns

The rows index the 88 patients, while the columns contain the 8 predictors.
We first briefly examine the data.



In [ ]:
%%R
print(BrainCancer['sex'].value_counts())
print(BrainCancer['diagnosis'].value_counts())
print(BrainCancer['status'].value_counts())

Before beginning an analysis, it is important to know how the
`status` variable has been coded.  Most software
uses the convention that `status == 1` indicates an
uncensored observation, and `status == 0` indicates a censored
observation. But some scientists might use the opposite coding. For
the `BrainCancer` data set 35 patients died before the end of
the study. Similarly, it is important to understand the coding of any
other qualitative variables: for instance, does `sex == 1` refer to
males or females?

To begin the analysis, we re-create a Kaplan-Meier survival curve shown in the book using the
`KaplanMeierFitter` from the `lifelines` package.
`tim` corresponds to $y_i$, the time to the $i$th event (either censoring or
death). The first argument to `fit` are the event times with the
second argument being the censoring variable with a 1 indicating an observed
failure time. The `plot` method plots a curve with pointwise confidence
intervals. By default, they produce 90% confidence intervals. This can be changed
by setting the `alpha` argument to 1 minus the desired
confidence level.



In [ ]:
%%R
from lifelines import KaplanMeierFitter
km = KaplanMeierFitter()
km_brain = km.fit(BrainCancer['tim'],
                  BrainCancer['status'])
km_brain.plot() #SUPPRESSOUTPUT

Next we create Kaplan-Meier survival curves that are stratified by
`sex`, in order to reproduce a second figure from the book.
We will store the grouped data frames to use below.



In [ ]:
%%R
by_sex = {}
for sex, df in BrainCancer.groupby('sex'):
    by_sex[sex] = df
    km_sex = km.fit(df['tim'],
                    df['status'])
    km_sex.plot(label='Sex=%d' % sex) #SUPPRESSOUTPUT

As discussed in the book, we can perform a
log-rank test to compare the survival of males to females. We use
the `logrank_test` function from the `lifelines.statistics` module.
The first two arguments are the event times, with the second
denoting the corresponding (optional) censoring indicators.



In [ ]:
%%R
from lifelines.statistics import logrank_test
print(logrank_test(by_sex[0]['tim'],
                   by_sex[1]['tim'],
                   by_sex[0]['status'],
                   by_sex[1]['status']))

The resulting $p$-value is $0.23$, indicating no evidence of a
difference in survival between the two sexes.

Next, we fit  Cox proportional hazards models using the `CoxPHFitter` estimator
from `lifelines`.
To begin, we consider a model that uses `sex` as the only predictor.



In [ ]:
%%R
from lifelines import CoxPHFitter
coxph = CoxPHFitter()
cox_fit = coxph.fit(BrainCancer,
                    'tim',
                    'status',
                    formula='sex')
cox_fit.summary[['coef', 'se(coef)', 'p']]

The first argument to `fit` should be a data frame containing
at least the event time (the second argument `tim` in this case)
as well as an optional censoring variable. The `formula` argument
allows for convenient specification of a design matrix. With
no `formula` argument, all columns besides the event time
and censoring varible are used in the design matrix. Note that you
may have issues with categorical variables without using the
`formula` argument.
It is possible to obtain the likelihood ratio test comparing this model to the one
with only an intercept as follows:



In [ ]:
%%R
cox_fit.log_likelihood_ratio_test()

Regardless of which test we use, we see that there is no clear
evidence for a difference in survival between males and females.  As
we learned in this chapter, the score test from the Cox model is
exactly equal to the log rank test statistic!

Now we fit a  model that makes use of additional predictors. We first note
that one of our `diagnosis` values is missing hence
we drop that observation before continuing. Also
note that not supplying a formula uses all columns in the regression.



In [ ]:
%%R
cleaned = BrainCancer.dropna()
formula = 'diagnosis + sex + loc + ki + gtv + stereo'
fit_all = coxph.fit(cleaned, 'tim', 'status', formula=formula)
fit_all.summary[['coef', 'se(coef)', 'p']]

The `diagnosis` variable has been coded so that the baseline
corresponds to meningioma, and a value of 2 corresponds to HG
glioma. The results indicate that the risk associated with HG glioma
is more than eight times (i.e. $e^{2.15}=8.62$) the risk associated
with meningioma. In other words, after adjusting for the other
predictors, patients with HG glioma have much worse survival compared
to those with meningioma.  In addition, larger values of the Karnofsky
index, `ki`, are associated with lower risk, i.e. longer survival.

Finally, we plot estimated survival curves for each diagnosis category,
adjusting for the other predictors.  To make these plots, we set the
values of the other predictors equal to the mean.  To avoid clutter in
the plots, we do not display confidence intervals.



In [ ]:
%%R
fit_all.plot_partial_effects_on_outcome('diagnosis',
                                        values=[0,1,2,3])

## Publication Data

The `Publication` data   can be
found in the `ISLR2` library.
We begin by plotting the Kaplan-Meier curves
stratified on the `posres` variable, which records whether the
study had a positive or negative result.



In [ ]:
%%R
Publication = load_data('Publication')
by_result = {}
for result, df in Publication.groupby('posres'):
    by_result[result] = df
    km_result = km.fit(df['tim'], df['status'])
    km_result.plot(label='Result=%d' % result) #SUPPRESSOUTPUT

As discussed previously, the $p$-values from fitting Cox’s
proportional hazards model to the `posres` variable are quite
large, providing no evidence of a difference in time-to-publication
between studies with positive versus negative results.



In [ ]:
%%R
posres_fit = coxph.fit(Publication,
                       'tim',
                       'status',
                       formula='posres')
posres_fit.summary[['coef', 'se(coef)', 'p']]

As expected, the log-rank test provides an identical conclusion.



In [ ]:
%%R
print(logrank_test(by_result[0]['tim'],
                   by_result[1]['tim'],
                   by_result[0]['status'],
                   by_result[1]['status']))

However, the results change dramatically when we include other
predictors in the model. Here we have excluded the funding mechanism
variable.



In [ ]:
%%R
formula = ('posres + multi + clinend ' +
          '+ sampsize + budget + impact')
coxph.fit(Publication,
          'tim',
   'status',
   formula=formula).summary[['coef', 'se(coef)', 'p']]

We see that there are a number of statistically significant variables,
including whether the trial focused on a clinical endpoint, the impact
of the study, and whether the study had positive or negative results.

## Call Center Data

In this section, we will simulate survival data using the relationship
between cumulative hazard and
the survival function explored in an exercise.
Our simulated data will represent the observed
wait times (in seconds) for 2,000 customers who have phoned a call
center.  In this context, censoring occurs if a customer hangs up
before his or her call is answered.

There are three covariates: `Operators` (the number of call
center operators available at the time of the call, which can range
from $5$ to $15$), `Center` (either A, B, or C), and
`Time` of day (Morning, Afternoon, or Evening). We generate data
for these covariates so that all possibilities are equally likely: for
instance, morning, afternoon and evening calls are equally likely, and
any number of operators from $5$ to $15$ is equally likely. We use the
`dmatrix` function from the `patsy` library introduced
in Chapter 3.



In [ ]:
%%R
from patsy import dmatrix
np.random.seed(10)
N = 2000
Operators = np.random.choice(np.arange(5, 16),
                             N,
                             replace=True)
Center = np.random.choice(['A', 'B', 'C'], 
                          N,
                          replace=True)
Time = np.random.choice(['Morn.', 'After.', 'Even.'], 
                        N,
                        replace=True)
D = pd.DataFrame({'Operators': Operators,
                  'Center': Center,
                  'Time': Time})		       
X = dmatrix("Operators + Center + Time",
            D,
            return_type='dataframe')
X = X.drop(['Intercept'], axis=1)

It is worthwhile to take a peek at the design matrix `X`, so
that we can be sure that we understand how the variables have been
coded.



In [ ]:
%%R
X[:5]

Next,  we specify the coefficients and the hazard function.



In [ ]:
%%R
true_beta = np.array([-0.3, 0, 0.2, -0.2, 0.04])
true_linpred = X.dot(true_beta)
hazard = lambda t: 1e-5 * t

To simulate the data we use a standard trick to simulate
random variables whose survival function is known, particularly
its inverse: if $S(t;x)$ is the survival function with
covariates $x$ then a random time satisfies

$$
S(T;x) \sim \text{Unif}(0,1)
$$


Using the relationship

$$
S(t;x) = \exp(-H(t;x))=\exp(-H_0(t)\exp(x^T\beta))
$$


we see that
$T$ at covariates $x$ can be simulated using the inverse
of the baseline cumulative hazard at that $x$. With $U \sim \text{Unif}(0,1)$

$$
T = H^{-1}_0(-\log(U) \exp(-x^T\beta))
$$


This simple function allows us to generate data
with a known inverse cumulative hazard baseline function:



In [ ]:
%%R
def sim_time(linpred, inv_cum_hazard, cutoff=1000):
    while True:
        U = np.random.sample()
        T = inv_cum_hazard(-np.log(U) /
                           np.exp(linpred))
        if T < cutoff:
            return T

cum_hazard = lambda t: 1e-5 * t**2 / 2
inv_cum_hazard = lambda v: np.sqrt(2 * v / 1e-5)

Here, we have set the coefficient associated with `Operators` to
equal $0.04$; in other words, each additional operator leads to a
$e^{0.04}=1.041$-fold increase in the “risk” that the call will be
answered, given the `Center` and `Time` covariates. This
makes sense: the greater the number of operators at hand, the shorter
the wait time! The coefficient associated with `Center == B` is
$-0.3$, and `Center == A` is treated as the baseline. This means
that the risk of a call being answered at Center B is 0.74 times the
risk that it will be answered at Center A; in other words, the wait
times are a bit longer at Center B.

We are now ready to generate data under the Cox proportional hazards
model. We’ll truncate the maximum time to 1000 seconds to keep
simulated wait times reasonable.



In [ ]:
%%R
D['y'] = np.array([sim_time(l, inv_cum_hazard)
            for l in true_linpred])

We know simulate our censoring variable, for which we assume
90% of calls were answered (`failed==1`) before the
customer hungup (`failed==0`).



In [ ]:
%%R
D['failed'] = np.random.choice([1, 0],
                                N,
                                p=[0.9, 0.1])
D[:5]

In [ ]:
%%R
D['failed'].mean()

We now plot  Kaplan-Meier survival curves. First, we stratify by `Center`



In [ ]:
%%R
by_center = {}
for center, df in D.groupby('Center'):
    by_center[center] = df
    km_center = km.fit(df['y'], df['failed'])
    km_center.plot(label='Center=%s' % center)
ax = plt.gca()
ax.set_title("Probability of Still Being on Hold") #SUPPRESSOUTPUT

Next, we stratify by `Time`.



In [ ]:
%%R
by_time = {}
for time, df in D.groupby('Time'):
    by_time[time] = df
    km_time = km.fit(df['y'], df['failed'])
    km_time.plot(label='Time=%s' % time)
ax = plt.gca()
ax.set_title("Probability of Still Being on Hold")  #SUPPRESSOUTPUT

It seems that calls at Call Center B take longer to be answered than
calls at Centers A and C. Similarly, it appears that wait times are
longest in the morning and shortest in the evening hours. We can use a
log-rank test to determine whether these differences are statistically
significant using `multivariate_logrank_test`.



In [ ]:
%%R
from lifelines.statistics import multivariate_logrank_test
print(multivariate_logrank_test(D['y'],
                                D['Center'],
                                D['failed']))
print(multivariate_logrank_test(D['y'],
                                D['Time'],
                                D['failed']))

As in the case of a categorical variable with 2 outcomes, these
results are similar to the likelihood ratio test
from the Cox proportional hazards model.



In [ ]:
%%R
print(coxph.fit(D, 'y', 'failed', formula='Center').log_likelihood_ratio_test())
print(coxph.fit(D, 'y', 'failed', formula='Time').log_likelihood_ratio_test())

We find that differences between centers are highly significant, as
are differences between times of day.

Finally, we fit Cox’s proportional hazards model to the data.



In [ ]:
%%R
formula = 'Operators + Time + Center'
fit_queuing = coxph.fit(D,
                        'y',
                        'failed',
                        formula=formula)
fit_queuing.summary[['coef', 'se(coef)', 'p']]

The $p$-values for `Center == B` and `Time == Even.`
are very small. It is also clear that the
hazard — that is, the instantaneous risk that a call will be
answered — increases with the number of operators. Since we
generated the data ourselves, we know that the true coefficients for
`Center == B`, `Center == C`,
`Time == Even.`, `Time == Morn.` and `Operators`  are $-0.3$,
$0$, $0.2$, and $-0.2$, $0.04$, respectively. The coefficient estimates
resulting from the Cox model are fairly accurate.